In [ ]:
pip install transformers
pip install InstructorEmbedding

In [1]:
pip install -r requirements.txt


  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.1
    Uninstalling protobuf-4.25.1:
      Successfully uninstalled protobuf-4.25.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.59.0 requires protobuf>=4.21.6, but you have protobuf 3.19.6 which is incompatible.
mysql-connector-python 8.2.0 requires protobuf<=4.21.12,>=4.21.1, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
web3 6.7.0 requires protobuf>=4.21.6, but you have protobuf 3.19.6 which is incompatible.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# python -m pip install --upgrade pip --user

In [46]:
pip install google-generativeai

   -------------------------------------- 146.6/146.6 kB 514.5 kB/s eta 0:00:00
   ---------------------------------------- 598.7/598.7 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 413.4/413.4 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.2.0 requires protobuf<=4.21.12,>=4.21.1, but you have protobuf 4.25.1 which is incompatible.
streamlit 1.22.0 requires protobuf<4,>=3.12, but you have protobuf 4.25.1 which is incompatible.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.1 which is incompatible.


In [45]:
from langchain.llms import GooglePalm

In [11]:
api_key = "COPY_GOOGLE_API_KEY"
llm = GooglePalm(google_api_key=api_key, temperature=0.8 )

In [15]:
llm("how are you ")

'doing fine'

In [16]:
from langchain.document_loaders.csv_loader import CSVLoader


In [18]:
loader = CSVLoader(file_path='Csv_file', source_column = "prompt")
data = loader.load()
data

[Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
 Document(page_content='prompt: Why should I trust Codebasics?\nresponse: Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.', metadata={'source': 'Why should I trust Codebasics?', 'row': 1}),
 Document(page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you ne

In [20]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

In [29]:
instuct_embedding = HuggingFaceInstructEmbeddings()

vectorsbd = FAISS.from_documents(documents=data, embedding=instuct_embedding)

load INSTRUCTOR_Transformer
max_seq_length  512


In [30]:
retriever = vectorsbd.as_retriever()
rdocs = retriever.get_relevant_documents("How many course do you have ?")

In [32]:
print(rdocs)

[Document(page_content='prompt: What business concepts and domains are covered in this course?\nresponse: We have covered the core functions such as Sales, Marketing, Finance, and Supply Chain with their fundamentals related to this course. The domain you will learn in this course is consumer goods which is projected to have more openings and high data analytics requirements at least until 2030.', metadata={'source': 'What business concepts and domains are covered in this course?', 'row': 32}), Document(page_content='prompt: What is the duration of this bootcamp? How long will it last?\nresponse: You can complete all courses in 3 months if you dedicate 2-3 hours per day.', metadata={'source': 'What is the duration of this bootcamp? How long will it last?', 'row': 8}), Document(page_content='prompt: Can I add this course to my resume? If Yes, how?\nresponse: Absolutely, we have a section in this course explaining how you can add the learnings from this course in your resume that will ap

In [33]:
from langchain.chains import RetrievalQA

In [34]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                    retriever = retriever, 
                                    input_key = "query", 
                                    return_source_documents = True )

In [35]:
chain("tell me about python course")

{'query': 'tell me about python course',
 'result': 'This course is designed to teach you the basics of Python programming language. It will cover the fundamentals of Python including variables, data types, operators, control flow statements, functions, modules and packages. You will also learn how to use Python to solve real-world problems.',
 'source_documents': [Document(page_content='prompt: What are the things I need to know before starting this course?\nresponse: This course is for absolute beginners hence you do not need any specific skills other than basic familiarity with computers', metadata={'source': 'What are the things I need to know before starting this course?', 'row': 17}),
  Document(page_content='prompt: What is different in this course compared to hundreds of courses on the internet and free tutorials on YouTube?\nresponse: Most of the courses available on the internet teach you how to build x & y without any business context and do not prepare you for real business

In [36]:
from langchain.prompts import PromptTemplate

In [37]:
template = """Given the following context and question, generate the answer based on the context only.
try to provide as much text as possible from "response" section in the source document context without make things up.
is not found in the context, kindly state "I dont know." Don't try to make up an answer.

CONTEXT : {context},
QUESTION : {question}

"""

In [39]:
prompt = PromptTemplate(template=template, input_variables= ["context", "question"])

In [41]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                    retriever = retriever, 
                                    input_key = "query", 
                                    return_source_documents = True,
                                    chain_type_kwargs = {"prompt":prompt})

In [44]:
chain("Do you have java script course")

{'query': 'Do you have java script course',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
